In [7]:
#import ctypes, os
#ctypes.CDLL("/usr/local/lib/libcholmod.so", mode=ctypes.RTLD_GLOBAL)
#import os, sys
#os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64:/usr/local/lib:/usr/local/lib:/usr/local/lib:/home/jcottaar/SuiteSparse/build/lib:/SuiteSparse/build/lib:/usr/local/cuda-12.5/lib64:"
# now import
from sksparse.cholmod import cholesky
import os, time, numpy as np, scipy.sparse as sp
from scipy.io import mmread
from sksparse.cholmod import cholesky

mtx = os.path.expanduser("~/nd6k.mtx")
A = mmread(mtx).tocsc()
A = sp.triu(A).tocsc()  # keep upper; CHOLMOD expects symmetric
n, nnz = A.shape[0], A.nnz
print(f"shape={A.shape}, nnz={nnz}")

# Sanity: this should be (18000, 18000) and ~3,457,658 nnz like the demo.
assert n > 1000, "Matrix looks tiny — are you sure this is nd6k.mtx?"

b = np.random.default_rng(0).standard_normal(n)

def run_once(use_gpu):
    os.environ["CHOLMOD_USE_GPU"] = "1" if use_gpu else "0"
    t0 = time.perf_counter(); F = cholesky(A); t1 = time.perf_counter()
    x = F(b); t2 = time.perf_counter()
    res = np.linalg.norm(A @ x - b) / (sp.linalg.norm(A)*np.linalg.norm(x) + np.linalg.norm(b))
    return t1-t0, t2-t1, res

# warmup + 3 steady timings
for mode in ("CPU","GPU"):
    ft, st, r = run_once(mode=="GPU")  # warmup
    times = [run_once(mode=="GPU")[0] for _ in range(3)]
    print(f"{mode}: factor times {', '.join(f'{t:.3f}s' for t in times)} | solve ~{st:.3f}s | resid {r:.2e}")

shape=(18000, 18000), nnz=3457658
CPU: factor times 0.020s, 0.019s, 0.020s | solve ~0.000s | resid 1.86e-03
GPU: factor times 0.023s, 0.020s, 0.021s | solve ~0.000s | resid 1.86e-03


In [2]:
# 1) Confirm dtype + structure (CHOLMOD expects float64 CSC, symmetric upper)
A.dtype, A.getformat(), A.shape, A.nnz

(dtype('float64'), 'csc', (18000, 18000), 3457658)

In [3]:
# 2) Force a full factorization and *materialize* L (this can't be ~10 ms)
from sksparse.cholmod import cholesky
import time
t0 = time.perf_counter()
F = cholesky(A)         # analyze + numeric factor
t1 = time.perf_counter()
L = F.L()               # build SciPy CSC copy of L (forces work if anything was lazy)
t2 = time.perf_counter()
t_factor = t1 - t0
t_materialize = t2 - t1
L.shape, L.nnz, t_factor, t_materialize

((18000, 18000), 18000, 0.015726660999462183, 0.0003520899999784888)

In [2]:
import sksparse.cholmod as c
print(c.__file__)

/home/jcottaar/miniconda3/envs/xlab/lib/python3.11/site-packages/sksparse/cholmod.cpython-311-x86_64-linux-gnu.so


In [7]:
os.environ["LD_LIBRARY_PATH"]

'/usr/local/cuda-12.5/lib64:'

In [2]:
import sksparse.cholmod as c, subprocess
print("sksparse ext:", c.__file__)
subprocess.run(["ldd", c.__file__])  # should point to /usr/local/lib/libcholmod.so

sksparse ext: /home/jcottaar/miniconda3/envs/xlab/lib/python3.11/site-packages/sksparse/cholmod.cpython-311-x86_64-linux-gnu.so
	linux-vdso.so.1 (0x00007ffde15e3000)
	libcholmod.so.5 => /usr/local/lib/libcholmod.so.5 (0x000074f5d2600000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x000074f5d2200000)
	libamd.so.3 => /usr/local/lib/libamd.so.3 (0x000074f5d2acf000)
	libcolamd.so.3 => /usr/local/lib/libcolamd.so.3 (0x000074f5d2ac6000)
	libcamd.so.3 => /usr/local/lib/libcamd.so.3 (0x000074f5d2ab8000)
	libccolamd.so.3 => /usr/local/lib/libccolamd.so.3 (0x000074f5d2aa9000)
	libopenblas.so.0 => /lib/x86_64-linux-gnu/libopenblas.so.0 (0x000074f5cfe20000)
	libcublas.so.12 => /usr/local/cuda/lib64/libcublas.so.12 (0x000074f5c9800000)
	libsuitesparseconfig.so.7 => /usr/local/lib/libsuitesparseconfig.so.7 (0x000074f5d2aa3000)
	libgomp.so.1 => /lib/x86_64-linux-gnu/libgomp.so.1 (0x000074f5d2a4d000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x000074f5d2964000)
	/lib64/ld-linux-x86-64.so.2 (0

CompletedProcess(args=['ldd', '/home/jcottaar/miniconda3/envs/xlab/lib/python3.11/site-packages/sksparse/cholmod.cpython-311-x86_64-linux-gnu.so'], returncode=0)

In [4]:
import sksparse.cholmod

AttributeError: module 'numpy' has no attribute 'dtype'

In [1]:
import numpy as np
import scipy.sparse as sp
from sksparse.cholmod import cholesky

# Small symmetric positive definite matrix
A_dense = np.array([
    [4.0, 1.0, 2.0],
    [1.0, 3.0, 0.5],
    [2.0, 0.5, 5.0]
])

# Convert to sparse CSC format (CHOLMOD prefers CSC)
A_sparse = sp.csc_matrix(A_dense)

# Right-hand side
b = np.array([1.0, 2.0, 3.0])

# Perform Cholesky factorization
factor = cholesky(A_sparse)

# Solve Ax = b
x = factor(b)

print("Solution x:", x)

# Optional: check residual
residual = A_dense @ x - b
print("Residual:", residual)

Solution x: [-0.22159091  0.63636364  0.625     ]
Residual: [0. 0. 0.]


In [3]:
import numpy

In [8]:
import os, time
import numpy as np
import scipy.sparse as sp
from scipy.io import mmread
from sksparse.cholmod import cholesky

# ---- config ----
MM_PATH = "/home/jcottaar/nd6k.mtx"          # adjust if your file lives elsewhere
USE_GPU = True                # toggle here

# CHOLMOD reads this at runtime
os.environ["CHOLMOD_USE_GPU"] = "1" if USE_GPU else "0"

# ---- load matrix ----
A = mmread(MM_PATH).astype(np.float64)   # COO
A = A.tocsc()

# Ensure SPD: if not symmetric, or has negatives on diag, build A^T A + eps*I
if (A - A.T).nnz != 0:
    A = (A.T @ A).tocsc()
# small diagonal shift for numerical stability
eps = 1e-10
A = A + sp.eye(A.shape[0], format="csc") * eps

# rhs (any dense vector works)
b = np.ones(A.shape[0], dtype=np.float64)

# ---- factorize & solve ----
t0 = time.time()
factor = cholesky(A)     # CHOLMOD factorization
x = factor(b)            # solve Ax=b
t1 = time.time()

# ---- checks ----
res = A @ x - b
print(f"CHOLMOD_USE_GPU={os.environ['CHOLMOD_USE_GPU']}")
print(f"n={A.shape[0]}, nnz={A.nnz}")
print(f"time: {t1 - t0:.3f} s")
print(f"||residual||_2 = {np.linalg.norm(res):.3e}")

CHOLMOD_USE_GPU=1
n=18000, nnz=6897316
time: 6.444 s
||residual||_2 = 3.205e-07
